In [1]:
# function fft(dir, l)
#     n = length(l)
#     @assert ispow2(n)
#     if (isone(n))
#         l
#     else
#         l1 = [v[i] for i in 1 : 2 : n ]
#         l2 = [v[i] for i in 2 : 2 : n ]
#         y0 = fft(dir, l1)
#         y1 = fft(dir, l2)
#         merge(dir, y0, y1)
#     end
# end

In [2]:
# w(dir, n. j) = ?

In [3]:
# function merge(dir, l1, l2)
#     n1 = length(l1)
#     n2 = length(l2)
#     @assert n1 == n2
#     n = n1 + n2
#     let rec mg l1 l2 j =
#         match (l1, l2) with
#         (x::xs, y::ys) ->
#             let z1 = mult (w dir n j, y) in
#             let zx = add (x, z1) in
#             let zy = sub (x, z1) in
#             let (a,b) = (mg xs ys (j+1)) in
#             (zx::a, zy::b)
#         | _ -> ([], []) in
#     let (a,b) = mg l1 l2 0
#          in (a @ b)
# end

In [4]:
function naive_dft(x)
    N = length(x)
    k = reshape(0:(N-1), 1, :)
    n = 0:(N-1)
    M = @. exp(-2 * im * π * k * n / N)
    M * x
end

naive_dft (generic function with 1 method)

In [5]:
using FFTW

In [6]:
a = rand(1024)
b = FFTW.fft(a)
c = naive_dft(a)
b ≈ c

true

In [7]:
function naive_dft2(x)
    @show N = length(x)
    @show k = reshape(0:(N-1), 1, :)
    n = 0:(N-1)
    M = @. exp(-2 * im * π * k * n / N)
    M * x
end

naive_dft2 (generic function with 1 method)

In [8]:
naive_dft2([10, 20, 30, 40])

N = length(x) = 4
k = reshape(0:N - 1, 1, :) = [0 1 2 3]


4-element Vector{ComplexF64}:
               100.0 + 0.0im
 -20.000000000000007 + 19.999999999999996im
               -20.0 - 9.797174393178823e-15im
 -19.999999999999982 - 20.00000000000001im

In [9]:
function dft_rec(a)
    n = length(a)
    @assert ispow2(n)
    isone(n) && return a
    n2 = n ÷ 2
    wn = exp(-2 * pi * im / n)

    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]
    y1 = dft_rec(a1)
    y2 = dft_rec(a2)

    wnk = [wn^(k - 1) for k in 1:n2]
    z1 = [y1[k] + wnk[k] * y2[k] for k in 1:n2]
    z2 = [y1[k] - wnk[k] * y2[k] for k in 1:n2]
    return vcat(z1, z2)
end

dft_rec (generic function with 1 method)

In [14]:
dft_rec([10, 20, 30, 40])

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [15]:
a = rand(1024)
b = FFTW.fft(a)
c = dft_rec(a)
b ≈ c

true